In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import html5lib
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException
import pandas as pd


In [42]:
df_mo =pd.read_csv("test.csv")
name = df_mo['영화명']
inp = name.values.tolist()
inp

driver = webdriver.Chrome('chromedriver')
driver.get("https://pedia.watcha.com/ko-KR")
driver.implicitly_wait(10)
result ={}

pop = driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div/div/div[2]/span[1]')
pop.click() 
but = WebDriverWait(driver, 1).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/header[1]/nav/div/div/ul/li[7]/button')))
but.click() 
ID = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div[2]/div/div/div/section/div/div/form/div[1]/label/div[1]/input')))
ID.click() 
ID.send_keys('cap345@nate.com')
passward = WebDriverWait(driver, 0.5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div[2]/div/div/div/section/div/div/form/div[2]/label/div[1]/input')))
passward.click() 
passward.send_keys('wonjae1496')
passward.send_keys(Keys.RETURN)

for title in tqdm(inp):
    #제목 검색
    time.sleep(1)
    elem = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.ID, 'desktop-search-field')))
    elem.click()   
    elem.send_keys(title)
    elem.send_keys(Keys.RETURN)
    driver.implicitly_wait(3)
    element_list = []
    i = 1
    review_list = []
    score_list = []
    try:
        #영화선택
        time.sleep(0.5)
        url = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div/div[1]/div/ul/li[1]/a')
        url.click()
        #더보기
        Url = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[2]/section[2]/header/div/div/a')
        Url.click()
        time.sleep(1)
        for k in range(250): #### 리뷰 갯수 지정하기 
            try:
                a = 'div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div > div > div > ul > div:nth-child(' + str(i) + ')'
                element = driver.find_element("css selector", a)
            except:
                if i == 250:
                    break
                driver.execute_script("window.scrollBy(0, window.innerHeight);")
            else:
                element_list.append(element)
                i += 1
        for iterable in element_list:
            try:
                score_data = iterable.find_element('css selector', 'div.css-jqudug.egj9y8a3 > div.css-31ods0.egj9y8a0 > span').text
                if score_data == '보고싶어요':
                    pass
                elif score_data == '보는 중':
                    pass
                else:
                    score_list.append(score_data)
                    reda = iterable.find_element('css selector', 'div.css-2occzs.egj9y8a1 > a > div > span').text
                    reda = reda.replace('\n','')
                    reda = reda.replace('#','')
                    review_list.append(reda)
            except:
                pass
            
        result[title] = review_list, score_list
    except NoSuchElementException:
        driver.get("https://pedia.watcha.com/ko-KR")

df_data = {'movie_id': [], 'reviews': [], 'score': []}

for movie_id, (reviews, score) in result.items():
    df_data['movie_id'].extend([movie_id] * len(score))
    df_data['reviews'].extend(reviews)
    df_data['score'].extend(score)

df = pd.DataFrame(df_data)
df = df[df['reviews'].str.strip() != ''] ### 스포일러 때문에 비어있는 리뷰 지우기
df['score'] = df['score'].apply(lambda x: float(x))
df['sentiment'] = df['score'].apply(lambda x: 1 if x > 2.5 else 0)
df.to_csv('test.csv', index=False, encoding='utf-8')

  0%|          | 0/2 [00:00<?, ?it/s]

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.125)
